# Add timestamps to DLC csv files where 'time' column alreday does not exist.

In [ ]:
import os
import cv2
import pandas as pd

# Directories
video_dir = r'C:\DeepLabCutProjects\DLC-Atanu-2024-12-25\Videos-to-analyze-temp'
analyzed_csv_dir = r'C:\DeepLabCutProjects\DLC-Atanu-2024-12-25\Analyzed-videos-filtered-temp'
output_dir = analyzed_csv_dir  # Overwrite in place

os.makedirs(output_dir, exist_ok=True)

for file in os.listdir(video_dir):
    if file.endswith('.mp4'):
        try:
            base_name = os.path.splitext(file)[0]
            video_path = os.path.join(video_dir, file)
    
            # Extract frame rate
            cap = cv2.VideoCapture(video_path)
            fps = cap.get(cv2.CAP_PROP_FPS)
            cap.release()
            
            # Match CSV file based on partial match
            matching_csv = [f for f in os.listdir(analyzed_csv_dir)
                            if base_name in f and f.endswith('_filtered.csv')]
            
            # Define csv_path
            if matching_csv:
                csv_file = matching_csv[0]
                csv_path = os.path.join(analyzed_csv_dir, csv_file)
    
                # Read CSV with multi-index header
                df = pd.read_csv(csv_path, header=[0, 1, 2], index_col=0)
                
                # Check if 'time' column already exists
                if ('time', '', 's') in df.columns:
                    print(f"Skipping (already has timestamp): {csv_file}")
                    with open('processing_log.txt', 'a') as log:
                        log.write(f"Skipping (already has timestamp): {csv_file}\n")
                    continue
        
                # Create timestamp column with matching MultiIndex structure
                time_column = pd.MultiIndex.from_tuples([('time', '', 's')])
                time_stamps = [round(i / fps, 3) for i in range(len(df))]
                time_df = pd.DataFrame(time_stamps, columns=time_column, index=df.index)
                
                # Combine and save
                df_out = pd.concat([time_df, df], axis=1)
                output_path = os.path.join(output_dir, csv_file)
                df_out.to_csv(output_path)
                print(f"✔ Timestamp added: {csv_file}")
                
            else:
                print(f"No matching CSV found for: {file}")
                with open('processing_log.txt', 'a') as log:
                    log.write(f"No matching CSV found for: {file}\n")

        except Exception as e:
            print(f"❌ Error processing {file}: {e}")
            with open('processing_log.txt', 'a') as log:
                log.write(f"❌ Error processing {file}: {e}\n")        
